In [1]:
import os
import json
import re
import scipy.io as spio
import numpy as np
from json_tricks.np import dump, dumps, load, loads
import pandas as pd
import pymworks
import cPickle as pkl
from collections import Counter

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

In [135]:
source = '/nas/volume1/2photon/projects'
experiment = 'gratings_phaseMod' #'retino_bar' #'gratings_phaseMod'
session = '20171025_CE062' #'20170902_CE054' #'20170825_CE055'
acquisition = 'FOV1' #'FOV1_zoom3x_run2' #'FOV1_planar'
functional_dir = 'functional'
acquisition_dir = os.path.join(source, experiment, session, acquisition)


In [136]:
# Load reference info:
ref_json = 'reference_%s.json' % functional_dir 
with open(os.path.join(acquisition_dir, ref_json), 'r') as fr:
    ref = json.load(fr)

# Load SI meta data:
si_basepath = ref['raw_simeta_path'][0:-4]
simeta_json_path = '%s.json' % si_basepath
with open(simeta_json_path, 'r') as fs:
    simeta = json.load(fs)

file_names = sorted([k for k in simeta.keys() if 'File' in k], key=natural_keys)
nfiles = len(file_names)


In [137]:
# Get PARADIGM INFO:
path_to_functional = os.path.join(acquisition_dir, functional_dir)
paradigm_dir = 'paradigm_files'
path_to_paradigm_files = os.path.join(path_to_functional, paradigm_dir)
path_to_raw = os.path.join(path_to_paradigm_files, 'raw')

# Get SERIAL data:
serialdata_fns = os.listdir(path_to_raw)
serialdata_fns = sorted([f for f in serialdata_fns if 'serial_data' in f], key=natural_keys)
print "Found %02d serial-data files, and %03d TIFFs." % (len(serialdata_fns), nfiles)

# Load MW info:
pydict_jsons = os.listdir(path_to_paradigm_files)
pydict_jsons = sorted([p for p in pydict_jsons if p.endswith('.json') and 'trial_info' in p], key=natural_keys)
print "Found %02d MW files, and %02d ARD files." % (len(pydict_jsons), len(serialdata_fns))

Found 10 serial-data files, and 010 TIFFs.
Found 10 MW files, and 10 ARD files.


In [138]:
serialdata_fns

['20171025_CE062_FOV1_zoom1x_file002_serial_data_20171025193326155599.txt',
 '20171025_CE062_FOV1_zoom1x_file003_serial_data_20171025194131517072.txt',
 '20171025_CE062_FOV1_zoom1x_file004_serial_data_20171025194452185873.txt',
 '20171025_CE062_FOV1_zoom1x_file005_serial_data_20171025194859992349.txt',
 '20171025_CE062_FOV1_zoom1x_file006_serial_data_20171025195215859707.txt',
 '20171025_CE062_FOV1_zoom1x_file009_serial_data_20171025200218237612.txt',
 '20171025_CE062_FOV1_zoom1x_file011_serial_data_20171025200932121894.txt',
 '20171025_CE062_FOV1_zoom1x_file012_serial_data_20171025201233466962.txt',
 '20171025_CE062_FOV1_zoom1x_file013_serial_data_20171025201549962462.txt',
 '20171025_CE062_FOV1_zoom1x_file014_serial_data_20171025202017691228.txt']

In [139]:
pydict_jsons

['trial_info_20171025_CE062_FOV1_zoom1x_file002.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_file003.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_file004.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_file005.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_file006.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_file009.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_file011.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_file012.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_file013.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_file014.json']

In [151]:
fid = 6
fn = serialdata_fns[fid]
print fn

20171025_CE062_FOV1_zoom1x_file011_serial_data_20171025200932121894.txt


In [152]:
trialdict_by_file = dict()
#for fid,fn in enumerate(sorted(serialdata_fns, key=natural_keys)):
# fid = 0
# fn = serialdata_fns[fid]
framerate = float(simeta['File001']['SI']['hRoiManager']['scanFrameRate'])

currfile = "File%03d" % int(fid+1)

print "================================="
print "Processing files:"
print "MW: ", pydict_jsons[fid]
print "ARD: ", serialdata_fns[fid]
print "---------------------------------"

### LOAD MW DATA.
with open(os.path.join(path_to_paradigm_files, pydict_jsons[fid]), 'r') as f:
    trials = json.load(f)

### LOAD SERIAL DATA.
ardata = pd.read_csv(os.path.join(path_to_raw, serialdata_fns[fid]), sep='\t')
print ardata.columns

frame_triggers = ardata[' frame_trigger']
arduino_time = ardata[' relative_arduino_time']
bitcodes = ardata[' pixel_clock']

frame_on_idxs = [idx+1 for idx,diff in enumerate(np.diff(frame_triggers)) if diff==1]
#print len(frame_on_idxs)
frame_on_idxs.append(0)
frame_on_idxs = sorted(frame_on_idxs)
print "Found %i frame-triggers:" % len(frame_on_idxs)

Processing files:
MW:  trial_info_20171025_CE062_FOV1_zoom1x_file011.json
ARD:  20171025_CE062_FOV1_zoom1x_file011_serial_data_20171025200932121894.txt
---------------------------------
Index([u'acquisition_trigger', u' frame_trigger', u' pixel_clock',
       u' abosolute_arduino_time', u' relative_arduino_time',
       u'     absolute_computer_time', u' relative_computer_time'],
      dtype='object')
Found 5630 frame-triggers:


In [155]:
[i for i,b in enumerate(bitcodes) if b==7][0]

6090

In [156]:
### Get bitcodes for each frame:
frame_bitcodes = dict()
for idx,frameidx in enumerate(frame_on_idxs):
    #framenum = 'frame'+str(idx)
    if idx==len(frame_on_idxs)-1:
        bcodes = bitcodes[frameidx:]
    else:
        bcodes = bitcodes[frameidx:frame_on_idxs[idx+1]]
    frame_bitcodes[idx] = bcodes

### Get first frame of trial start:
modes_by_frame = dict()
for frame in frame_bitcodes.keys():
    bitcode_counts = Counter(frame_bitcodes[frame])
    modes_by_frame[frame] = bitcode_counts.most_common(1)[0][0]


In [157]:
[k for k in sorted(modes_by_frame.keys()) if modes_by_frame[k]>0]

[272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 324,
 325,
 326,
 327,
 328,
 330,
 331,
 332,
 333,
 334,
 336,
 337,
 338,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 352,
 353,
 354,
 355,
 357,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 424,
 425,
 426,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 434,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453

In [170]:
if 'grating' in experiment:
    first_stim_frame = [k for k in sorted(modes_by_frame.keys()) if modes_by_frame[k]>0][0]
else:
    first_stim_frame = [k for k in sorted(modes_by_frame.keys()) if modes_by_frame[k]>0][1]

print first_stim_frame


272


In [171]:

### Get all bitcodes and corresonding frame-numbers for each trial:
trialdict = dict()
allframes = sorted(frame_bitcodes.keys()) #, key=natural_keys)
curr_frames = sorted(allframes[first_stim_frame+1:]) #, key=natural_keys)


In [173]:
print first_stim_frame
frame_bitcodes[first_stim_frame]

272


6095    7
6096    7
6097    7
6098    7
6099    7
6100    7
6101    7
6102    7
6103    7
6104    7
6105    7
6106    7
6107    6
6108    6
6109    6
6110    6
6111    6
6112    6
6113    6
6114    6
6115    6
6116    6
Name:  pixel_clock, dtype: int64

In [174]:
trials[trial]['iti_duration']/1E3 # * framerate

KeyError: u'20'

In [175]:

for trial in sorted(trials.keys(), key=natural_keys): #sorted(trials.keys, key=natural_keys):
    #print trial
    trialdict[trial] = dict()
    trialdict[trial]['name'] = trials[trial]['stimuli']['stimulus']
    trialdict[trial]['duration'] = trials[trial]['stim_off_times'] - trials[trial]['stim_on_times']

    if int(trial)>1:
    # Skip a good number of frames from the last "found" index of previous trial.
    # Since ITI is long (relative to framerate), this is safe to do. Avoids possibility that
    # first bitcode of trial N happened to be last stimulus bitcode of trial N-1
        nframes_to_skip = int(((trials[trial]['iti_duration']/1E3) * framerate) - 5)
        #nframes_to_skip = int(((1000/1E3) * framerate) - 5)
        print 'skipping iti...', nframes_to_skip
        curr_frames = allframes[first_frame+nframes_to_skip:]

    first_found_frame = []
    minframes = 4
    for bitcode in trials[trial]['all_bitcodes']:
            looking = True
            while looking is True:
                for frame in sorted(curr_frames):
                    tmp_frames = [i for i in frame_bitcodes[frame] if i==bitcode]
                    consecutives = [i for i in np.diff(tmp_frames) if i==0]

                    if frame>1:
                        tmp_frames_pre = [i for i in frame_bitcodes[int(frame)-1] if i==bitcode]
                        consecutives_pre = [i for i in np.diff(tmp_frames_pre) if i==0]
                    
                    if len(trials[trial]['all_bitcodes'])<3:
                        if len(consecutives)>=minframes:
                            first_frame = frame
                            looking = False
                    else:
                        if frame>1 and len(consecutives_pre)>=minframes:
                            if len(consecutives_pre) > len(consecutives):
                                first_frame = int(frame) - 1
                            elif len(consecutives)>=minframes:
                                first_frame = int(frame)
                            #print "found2...", bitcode, first_frame #len(curr_frames)
                            looking = False

                        elif len(consecutives)>=minframes:
                            first_frame = frame 
                            #print "found...", bitcode, first_frame #len(curr_frames)
                            looking = False

                    if looking is False:
                        break

            first_found_frame.append((bitcode, first_frame)) #first_frame))
            curr_frames = allframes[first_frame+1:] #curr_frames[idx:] #curr_frames[first_frame:]

    #if (first_found_frame[-1][1] - first_found_frame[0][1])/framerate > 2.5:
    #print "Trial %i dur (s):" % int(trial)
    print (first_found_frame[-1][1] - first_found_frame[0][1])/framerate, '[Tial %i]' % int(trial)

    trialdict[trial]['stim_on_idx'] = first_found_frame[0][1]
    trialdict[trial]['stim_off_idx'] = first_found_frame[-1][1]

3.98808950348 [Tial 1]
skipping iti... 262
4.01049450069 [Tial 2]
skipping iti... 262
3.98808950348 [Tial 3]
skipping iti... 262
3.98808950348 [Tial 4]
skipping iti... 262
4.01049450069 [Tial 5]
skipping iti... 262
4.01049450069 [Tial 6]
skipping iti... 262
3.98808950348 [Tial 7]
skipping iti... 262
4.01049450069 [Tial 8]
skipping iti... 262
4.01049450069 [Tial 9]
skipping iti... 262
4.01049450069 [Tial 10]
skipping iti... 262
4.01049450069 [Tial 11]
skipping iti... 262
4.01049450069 [Tial 12]


In [120]:
trials[trial]

{u'all_bitcodes': [14, 1],
 u'end_time_ms': 2616178091.0,
 u'iti_bitcode': 1,
 u'iti_duration': 4000,
 u'start_time_ms': 2616173074.0,
 u'stim_bitcode': 14,
 u'stim_off_times': 15100.0,
 u'stim_on_times': 14084.0,
 u'stimuli': {u'position': [0.0, 0.0], u'scale': 65.0, u'stimulus': u'10'}}

In [121]:
first_found_frame

[(14, 856), (1, 901)]

In [95]:
(first_found_frame[1][1] - first_found_frame[0][1]) / framerate

1.0082248744760032

In [118]:
frame_bitcodes[677]

15175     0
15176    14
15177     3
15178     1
15179     1
15180     1
15181     1
15182     1
15183     1
15184     1
15185     1
15186     1
15187     1
15188     1
15189     1
15190     1
15191     1
15192     1
15193     1
15194     1
15195     1
15196     1
Name:  pixel_clock, dtype: int64

In [36]:
path_to_paradigm_files

'/nas/volume1/2photon/projects/gratings_phaseMod/20170927_CE059/FOV1_zoom3x/functional_subset/paradigm_files'

In [38]:
with open(os.path.join(path_to_paradigm_files, 'parsed_trials.pkl'), 'rb') as f:
    trialdict = pkl.load(f)

In [94]:
ref['frame_idxs'].index(2403) + 1

1443

In [50]:
ref['frame_idxs'][0:5]

[1, 2, 3, 6, 7]